In [1]:
import uqcreator as uqc

In [2]:
settings = uqc.Settings()

settings.mesh['file'] = 'naca_profile.su2'
settings.mesh['bc'] = [["farfield","dirichlet"], ["airfoil", "noslip"]]

settings.problem['uncertainties'] = [["Legendre"],[0.5]]
settings.problem['cfl'] = 0.9
settings.problem['tEnd'] = 1.0

settings.moment_system['moments'] = [["totalDegree"],[8]]
settings.moment_system['quadPoints'] = [["tensorizedCCGrid"],[4]]

In [10]:
problem = uqc.Euler()
def IC(self, x, xi):
    gamma       = 1.4
    R           = 287.87
    T           = 273.15
    Ma          = 0.8
    AoA         = 1.25
    p           = 101325.0
    rhoFarfield = p / ( R * T )

    sigma = np.ones(2)

    if( len(xi) > 1 ):
        Ma = Ma - sigma[1]
        Ma = Ma + xi[1] * sigma[1]

    a = np.sqrt( gamma * R * T )
    uMax  = Ma * a
    angle = ( AoA + sigma[0] * xi[0] ) * ( 2.0 * np.pi ) / 360.0
    uF    = uMax * np.cos( angle )
    vF    = uMax * np.sin( angle )

    kineticEnergy = 0.5 * rhoFarfield * ( uF**2 +  vF**2 )
    innerEnergy   = ( p / ( rhoFarfield * ( gamma - 1 ) ) ) * rhoFarfield
    
    y = np.zeros(self.nStates)
    y[0] = rhoFarfield
    y[1] = rhoFarfield * uF
    y[2] = rhoFarfield * vF    
    y[3] = kineticEnergy + innerEnergy
    return y
    
problem.IC = IC

In [11]:
closure = uqc.StochasticGalerkin(problem.nStates)

In [8]:
solver = uqc.Solver(settings, problem, closure)
solver.run()

SyntaxError: invalid syntax (<string>, line 1)